In [ ]:
# Code to plot the HI Mass as a function of Halo Mass (Mvir)
# First off, import the packages we need
import numpy as np  # work horse package for numerical work in python
import matplotlib.pyplot as plt  # plotting library
import pandas as pd
from dragons import meraxes, munge
import random
import sys
from Functions import *
import statistics 
import seaborn as sns

#get gals, snaps, sims_props
snapshot_used = 250 # z = 0

#gals, sim_props = meraxes.read_gals(fname_in, snapshot_used, sim_props=True, pandas=True)
gals, sim_props, snaplist = get_gal_catalogue(snapshot_used)
snaplist_index_used = list(snaplist[0]).index(snapshot_used)# convert to list and find the index of the snap we are using
redshift_used = snaplist[1][snaplist_index_used] # Get the redshift for corresponding snapshot
redshift_used = str(round(redshift_used, 1)) # round float to nearest whole int

print("snaps available: ", snaplist[0])
# Log the masses : CONERT TO PROPER UNITS
gals["HIMass"] = np.log10(gals["HIMass"]*1e10)
gals["Mvir"] = np.log10(gals["Mvir"]*1e10)

#gals.describe()# summarise the stats of HIMass
#gals_ran.shape # returns number of galaxies
#gals.dtypes #returns types of available properties
#gals.columns #returns all avaiable properties of galxies

# remove infs
gals[gals.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]
# drop nan values
gals.dropna()

In [ ]:
# Split the data into 1000 quantiles (varies bin width to keep same number of data points in each bin)
nbins = 100
gals["Mvir_group"] = pd.cut(gals["Mvir"], bins=nbins)

# Group galaxies by Mvir_group 
grouped = gals.groupby("Mvir_group")

# Get parameters of each group and save as results_df
results_df_HIMass = grouped["HIMass"].agg(["mean", "median", "std", "size"])
results_df_Mvir = grouped["Mvir"].agg(["mean", "median", "std", "size"])
#results_df_HIMass.sort_values(by='size', ascending=False)
# drop last row 
#results_df_Mvir.drop(results_df_Mvir.tail(1).index,inplace=True) # drop last n rows
#results_df_HIMass.drop(results_df_HIMass.tail(1).index,inplace=True) # drop last n rows

Mvir_mean = results_df_Mvir["mean"]
Mvir_std = results_df_Mvir["std"]

HIMass_median = results_df_HIMass['median']
HIMass_std = results_df_HIMass["std"]


In [ ]:
# Now plot the simulation for Cen, Sat and All and the Zwaan 2005 and Martin 2010 dataset on one fig
fig, ax = plt.subplots(1, 1)

label_z = ("z = " + redshift_used)

plot = ax.scatter(Mvir_mean, HIMass_median, color= "purple", marker= "*", s=2, label=label_z)  
ax.set_xlabel(r"$\log_{10}(M_{h}) [{\rm M_{\odot}}]$")
ax.set_ylabel(r"$\log_{10}(M_{HI}) [{\rm M_{\odot}}]$")
ax.set_ylim([0,15])
ax.set_xlim([9.5,15.5])
# Add the legend
plt.legend(title=("DRAGONS"),
           loc='upper left',
           ncol=5,
           fontsize=12)

# Finally save the figure as a PDF
plt.tight_layout()
nbins = str(nbins)
name_of_plot=("Alice_MHI_Mvir_" + nbins + "_scatter")
plt.savefig("plots/" + name_of_plot + ".pdf")

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_xlabel(r"$\log_{10}(M_{h}) [{\rm M_{\odot}}]$")
ax.set_ylabel(r"$\log_{10}(M_{HI}) [{\rm M_{\odot}}]$")
ax.set_ylim([6,15])
ax.set_xlim([9.5,15.5])
label_sigma = ("$1\sigma$")
color_used=np.random.rand(3,)
ax.plot(Mvir_mean, HIMass_median, color=color_used, label=label_z)
ax.fill_between(Mvir_mean, 
                HIMass_median - HIMass_std, 
                HIMass_median + HIMass_std, 
                alpha=0.2, color =color_used, label=label_sigma)

plt.legend(title=("DRAGONS"),
           loc='upper left',
           fontsize=12)

name_of_plot=("Alice_MHI_Mvir_" + nbins + "_shaded_z-" + redshift_used)
plt.savefig("plots/" + name_of_plot + ".pdf")